# PAI for all sites in Berchtesgaden and Bosland based on 30 degree zenith cones

This notebook generates different vegetation structure values such as PAI at 50 m height from ground from rxp and rdbx files of any scan position.

## Load all the required modules

In [ ]:
import os
import csv
import numpy as np
import glob

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MaxNLocator

from pylidar_tls_canopy import riegl_io, plant_profile, grid

import pandas as pd
from pathlib import Path
import shutil
import timeit
os.chdir(f'/mnt/l/lab_nas/projects/weave/data/raw/')
cwd = os.getcwd()
print("Current Working Directory:", cwd)

Current Working Directory: /mnt/l/lab_nas/projects/weave/data/raw


Create csv with all rxp and rdbx files of all scans conducted at TOMST positions (the current code snippet runs very slow at ~1.5 hours)

In [ ]:
# Define the root directory where your files are stored
root_dir = r'/mnt/l/lab_nas/projects/weave/data/raw/'

# List to store the extracted path components
data = []

# Walk through the directory structure
for dirpath, _, filenames in os.walk(root_dir):
    for file in filenames:
        # Process only .rxp and .rdbx files
        if file.endswith((".rxp", ".rdbx")):
            full_path = os.path.join(dirpath, file)

            # Split the path into components
            path_parts = full_path.split(os.sep)  # OS-independent separator

            # Append full path as the last column
            path_parts.append(full_path)

            # Store in data list
            data.append(path_parts)

# Define the CSV file name
csv_filename = "output.csv"

# Write to CSV
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)

    # Create header dynamically (plus "Full_Path" column)
    header = [f"Level_{i}" for i in range(len(data[0]) - 1)] + ["Full_Path"]
    writer.writerow(header)

    # Write data
    writer.writerows(data)

print(f"CSV file '{csv_filename}' created successfully.")


Current Working Directory: /mnt/l/lab_nas/projects/weave/data/raw
CSV file 'output.csv' created successfully.


Clean the csv

In [8]:
# Load the CSV file
file_path = "output.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Drop the first four columns
df = df.iloc[:, 8:]

# Rename the columns
new_column_names = ["Site", "Plot", "Position",
                    "Scanner", "Year", "Orientation",
                    "File", "Path"]
df.columns = new_column_names[: len(df.columns)]

# Save the modified CSV file (optional)
df.to_csv("output_cleaned.csv", index=False)

# Display the first few rows
print(df.head())

  Site  Plot Position Scanner  Year Orientation                File  \
0  BGD     1      C03  vz400i  2023        hori  230709_150710.rdbx   
1  BGD     1      C03  vz400i  2023        hori   230709_150710.rxp   
2  BGD     1      C03  vz400i  2023        vert  230709_150508.rdbx   
3  BGD     1      C03  vz400i  2023        vert   230709_150508.rxp   
4  BGD     1      C05  vz400i  2023        hori  230710_092258.rdbx   

                                                Path  
0  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...  
1  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...  
2  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...  
3  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...  
4  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...  


In [9]:
def get_filepaths(data):
    try:
        # Group by 'folder' and aggregate paths
        grouped = data.groupby(['Site',
                              'Plot',
                              'Position',
                              'Year']).agg({
            'Path': list
        }).reset_index()
        
        # Assign paths to rdbx and rxp files
        grouped['rdb_v'] = grouped['Path'].apply(lambda x: next((f for f in x if f.endswith(".rdbx") and '/vert/' in f), None))
        grouped['rxp_v'] = grouped['Path'].apply(lambda x: next((f for f in x if f.endswith(".rxp") and '/vert/' in f), None))
        grouped['dat_v'] = "/mnt/l/lab_nas/projects/weave/data/tls/matrices/ScanPosDummy.DAT"
        grouped['rdb_h'] = grouped['Path'].apply(lambda x: next((f for f in x if f.endswith(".rdbx") and '/hori/' in f), None))
        grouped['rxp_h'] = grouped['Path'].apply(lambda x: next((f for f in x if f.endswith(".rxp") and '/hori/' in f), None))
        grouped['dat_h'] = "/mnt/l/lab_nas/projects/weave/data/tls/matrices/ScanPosDummy.DAT"
        
        return grouped[['rdb_v', 'rxp_v', 'dat_v',
                        'rdb_h', 'rxp_h', 'dat_h']]
    
    except Exception as e:
        print(f"Error processing data '{data}': {e}")
        return [None, None, None, None, None, None]

In [10]:
cwd = os.getcwd()
print(cwd)
file_list = get_filepaths(df).apply(pd.Series)
print(file_list.head())
print(len(file_list))
file_list.to_excel('scan_info.xlsx', index=False)

/mnt/l/lab_nas/projects/weave/data/raw
                                               rdb_v  \
0  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
1  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
2  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
3  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
4  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   

                                               rxp_v  \
0  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
1  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
2  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
3  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   
4  /mnt/l/lab_nas/projects/weave/data/raw/BGD/001...   

                                               dat_v  \
0  /mnt/l/lab_nas/projects/weave/data/tls/matrice...   
1  /mnt/l/lab_nas/projects/weave/data/tls/matrice...   
2  /mnt/l/lab_nas/projects/weave/data/tls/matrice...   
3  /mnt/l/lab_nas/projects/weave/data/tls/matrice...   
4  /mnt

In [11]:
def get_coneprofiles(scans):
    vert_rdbx_fn = scans['rdb_v']
    vert_rxp_fn = scans['rxp_v']
    vert_transform_fn = scans['dat_v']

    hori_rdbx_fn = scans['rdb_h']
    hori_rxp_fn = scans['rxp_h']
    hori_transform_fn = scans['dat_h']
    print("done1")
    # Determine the origin coordinates to use
    transform_matrix = riegl_io.read_transform_file(vert_transform_fn)
    x0,y0,z0,_ = transform_matrix[3,:]

    grid_extent = 60
    grid_resolution = 1
    grid_origin = [x0,y0]
    

    # If using RXP files only as input, set rxp to True:
    x,y,z,r = plant_profile.get_min_z_grid([vert_rdbx_fn], 
                                           [vert_transform_fn], 
                                        grid_extent, grid_resolution, grid_origin=grid_origin,
                                        rxp=False)
    
    # Optional weighting of points by 1 / range
    planefit = plant_profile.plane_fit_hubers(x, y, z, w=1/r)
    #planefit['Summary']
    
    print("done2")
    # If the ground plane is not defined then set ground_plane to None
    # and use the sensor_height argument when adding scan positions
    vpp = plant_profile.Jupp2009(hres=0.5, zres=30, ares=90, 
                                min_z=0, max_z=30, min_h=0, max_h=50,
                                ground_plane=planefit['Parameters'])

    # If using RXP files only as input, set rdbx_file to None (the default)
    query_str = ['reflectance > -20']
    print("done3")

    # If using RXP files only as input, set rdbx_file to None (the default)
    query_str = ['reflectance > -20']
    vpp.add_riegl_scan_position(hori_rxp_fn, hori_transform_fn, sensor_height=1.8,
        rdbx_file=hori_rdbx_fn, method='WEIGHTED', min_zenith=0, max_zenith=30,
        query_str=query_str)
    
    vpp.get_pgap_theta_z(min_azimuth=0, max_azimuth=360)
    pgap_cone = vpp.pgap_theta_z
    #pai_weighted_cone = vpp.calcSolidAnglePlantProfiles()
    #pai_linear_cone = vpp.calcLinearPlantProfiles()
    #weighted_pai = vpp.calcSolidAnglePlantProfiles()
    #linear_pai = vpp.calcLinearPlantProfiles()

    #hinge_pavd = vpp.get_pavd(hinge_pai)
    #linear_pavd = vpp.get_pavd(linear_pai)
    #weighted_pavd = vpp.get_pavd(weighted_pai)
    
    print("done4")

    #weighted_pai= pd.DataFrame(weighted_pai)
    #weighted_pai.to_csv(str(up)+'pai'+'.csv', sep=',', index=False, encoding='utf-8')

    #weighted_pavd= pd.DataFrame(weighted_pavd)
    #weighted_pavd.to_csv(str(up)+'pavd'+'.csv', sep=',', index=False, encoding='utf-8')
    
    print(pgap_cone)
    return [pgap_cone]

In [ ]:
bgd001_cones = file_list.head(5).apply(get_coneprofiles, axis=1)
bgd001_cones.to_csv('pgap_cone_allscans_unfiltered.csv', index=False)

done1
